testing Torch backend:
* circuit : oscillator, transmon, fluxonium

In [314]:
from DiSuQ.Torch import models
from torch import tensor,complex
from DiSuQ.Torch.components import *
from DiSuQ.Torch.dense import basisQq,transformationMatrix,chargeStates,fluxStates,outer,basisFf
from numpy import arange,log10,sqrt,diff,around,diff,pi
from DiSuQ.utils import plotCompare
from DiSuQ.Torch.circuit import hamiltonianEnergy
from torch import set_num_threads
set_num_threads(32)

### LC oscillator

In [337]:
El = 1.2
Ec = .054
size = 50
circuit = models.oscillatorLC([size],El=El,Ec=Ec,sparse=False)

In [338]:
circuit.circuitState()

{'L': 1.1999996900558472, 'C': 0.05399996042251587}

In [340]:
sqrt(8*El*Ec)

0.72

In [341]:
.25/2/indEnergy(El) # upper bound on potential energy

5.921762640653615

In [339]:
H_LC = circuit.oscillatorHamiltonianLC()
oscillator = circuit.circuitEnergy(H_LC=H_LC,grad=False)

In [342]:
H_LC = circuit.fluxHamiltonianLC()
flux = circuit.circuitEnergy(H_LC=H_LC,grad=False)

In [343]:
H_LC = circuit.chargeHamiltonianLC()
charge = circuit.circuitEnergy(H_LC=H_LC,grad=False)

In [351]:
plotCompare(arange(len(oscillator)),{'oscillator':oscillator,'charge':charge,
                                     'flux':flux},'Energy Spectrum','levels',r'$$\Delta E(GHz)$$')

## comparison : Capacitive :: Inductive

In [10]:
from DiSuQ.Torch import components
from DiSuQ.Torch import circuit
from DiSuQ.Torch import dense

In [11]:
components.C(0,1,Ec=10).capacitance(), components.L(0,1,El=.025).inductance()

(tensor(0.0500, grad_fn=<MulBackward0>),
 tensor(1.0132, grad_fn=<MulBackward0>))

In [12]:
components.indEnergy(.025)

1.0132118364233778

In [13]:
components.capEnergy(4050)

0.0001234567901234568

In [14]:
cap = circuit.hamiltonianEnergy(dense.basisQq(100)@dense.basisQq(100)/2/4050)

In [15]:
ind = circuit.hamiltonianEnergy(dense.basisFf(100)@dense.basisFf(100)/2/.025)

In [16]:
(cap/ind)[:5]

tensor([   nan, 0.9976, 0.9976, 0.9976, 0.9976])

In [17]:
El = 1e-5
Ec = 100
circuit = models.oscillatorLC([500],El=El,Ec=Ec,sparse=False)

In [18]:
H_LC = circuit.chargeHamiltonianLC()
charge = circuit.circuitEnergy(H_LC=H_LC,grad=False)

In [19]:
charge[:3]

array([1.6449265e-04, 4.0000003e+02, 4.0000009e+02], dtype=float32)

In [20]:
Ec = 1e-5
El = 1000
circuit = models.oscillatorLC([500],El=El,Ec=Ec,sparse=False)

In [21]:
H_LC = circuit.fluxHamiltonianLC()
flux = circuit.circuitEnergy(H_LC=H_LC,grad=False)

In [22]:
N = 10
plotCompare(arange(N),{'Ec':log10(cap),'El':log10(ind)})

/tmp/ipykernel_462461/3138489726.py:2: RuntimeWarning:

divide by zero encountered in log10



### Transmon

In [290]:
Ej,Ec = 20,.03
size = 50

In [291]:
basis = {'O':[],'I':[],'J':[size]}
circuit = models.transmon(basis,Ej,Ec,sparse=False)
H_LC = circuit.kermanHamiltonianLC()
H_J = circuit.kermanHamiltonianJosephson
kerman = circuit.circuitEnergy(H_LC=H_LC,H_J=H_J)

In [292]:
circuit = models.transmon([size],Ej,Ec,sparse=False)
H_LC=circuit.chargeHamiltonianLC()
H_J=circuit.josephsonCharge
charge = circuit.circuitEnergy(H_LC=H_LC,H_J=H_J)

In [293]:
circuit = models.transmon([size],Ej,Ec,sparse=False)
H_LC=circuit.fluxHamiltonianLC()
H_J=circuit.josephsonFlux
flux = circuit.circuitEnergy(H_LC=H_LC,H_J=H_J)

In [294]:
kerman = kerman.detach().numpy()
charge = charge.detach().numpy()
flux = flux.detach().numpy()
plotCompare(arange(len(kerman)),{'kerman':kerman,'charge':charge,'flux':flux},'Energy Spectrum','levels','E(GHz)')

## Fluxonium

In [27]:
basis = {'O':[1000],'I':[],'J':[]}
circuit = models.fluxonium(basis,Ej=100,sparse=False)

In [28]:
flux_profile = tensor(arange(0,1,.01))
flux_profile = [[flux] for flux in flux_profile]
flux_point = ('I')

In [ ]:
H_LC = circuit.kermanHamiltonianLC()
H_J = circuit.kermanHamiltonianJosephson
E0,(EI,EII) = circuit.spectrumManifold(flux_point,flux_profile,H_LC,H_J,excitation=[1,2])

In [ ]:
EI = tensor(EI).detach().numpy()
EII = tensor(EII).detach().numpy()

In [ ]:
plotCompare(arange(0,1,.01),{'Ist Excitation':EI,'IInd Excitation':EII,'Anharmonicity':EII-EI-EI},
            'Fluxonium-Excitation','external flux','E(GHz)')